# Previsão de Sucesso de Startups - Modelo Gradient Boosting
Este notebook apresenta um pipeline completo para prever o sucesso de startups utilizando Gradient Boosting, conforme as regras do desafio. Todas as etapas estão documentadas em português para facilitar o entendimento e a avaliação.

## Pipeline do Projeto
1. **Importação das bibliotecas**: Apenas bibliotecas permitidas pelo regulamento (numpy, pandas, scikit-learn, matplotlib, seaborn).
2. **Carregamento dos dados**: Leitura dos arquivos de treino, teste e submissão.
3. **Tratamento de valores ausentes**: Imputação por mediana (numéricas) e moda (categóricas).
4. **Codificação de variáveis categóricas**: One-Hot Encoding para `category_code`.
5. **Seleção de features**: Escolha das variáveis mais relevantes para o modelo.
6. **Padronização dos dados**: Uso de StandardScaler para normalizar as features.
7. **Divisão dos dados**: Separação em treino e validação para avaliação do modelo.
8. **Treinamento do modelo**: Utilização do Gradient Boosting para classificação binária.
9. **Otimização de hiperparâmetros**: Busca dos melhores parâmetros com GridSearchCV.
10. **Avaliação do modelo**: Métricas de acurácia, precisão, recall, F1-score e matriz de confusão.
11. **Geração do arquivo de submissão**: Previsão no conjunto de teste e exportação do arquivo final.

### Sobre o desafio
O objetivo é prever se uma startup será bem-sucedida (ativa/adquirida) ou não (fechada) a partir de variáveis como captação de recursos, setor, localização e conexões estratégicas. O modelo deve atingir alta acurácia e seguir as regras do campeonato, utilizando apenas as bibliotecas permitidas e os dados fornecidos.

In [56]:
# Importação das bibliotecas 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [57]:
# Carregamento dos dados
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')
print('Train shape:', train.shape)
print('Test shape:', test.shape)
train.head()

Train shape: (646, 33)
Test shape: (277, 32)


,id,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,is_CA,...,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,labels
0,719,10.42,13.09,8.98,12.72,4,3,4087500,3,1,...,0,0,1,1,0,0,0,0,1.0,0
1,429,3.79,3.79,NaN,NaN,21,1,45000000,0,0,...,0,0,0,0,0,1,0,0,1.0,1
2,178,0.71,2.28,1.95,2.28,5,2,5200000,2,1,...,0,1,1,0,1,0,0,0,1.0,0
3,197,3.00,5.00,9.62,10.39,16,2,14500000,2,0,...,0,0,0,1,0,1,0,0,2.0,1
4,444,0.66,5.88,6.21,8.61,29,5,70000000,4,1,...,0,0,0,0,1,1,1,1,2.8,1


In [58]:
# Tratamento de missing values
num_cols = [col for col in train.select_dtypes(include=['float64', 'int64']).columns if col != 'labels']
cat_cols = train.select_dtypes(include=['object']).columns
imputer_num = SimpleImputer(strategy='median')
imputer_cat = SimpleImputer(strategy='most_frequent')
train[num_cols] = imputer_num.fit_transform(train[num_cols])
test[num_cols] = imputer_num.transform(test[num_cols])
train[cat_cols] = imputer_cat.fit_transform(train[cat_cols])
test[cat_cols] = imputer_cat.transform(test[cat_cols])

In [59]:
# Codificação da variável categórica principal
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_train = encoder.fit_transform(train[['category_code']])
encoded_test = encoder.transform(test[['category_code']])
encoded_cols = encoder.get_feature_names_out(['category_code'])
train_encoded = pd.DataFrame(encoded_train, columns=encoded_cols, index=train.index)
test_encoded = pd.DataFrame(encoded_test, columns=encoded_cols, index=test.index)
train = pd.concat([train.drop('category_code', axis=1), train_encoded], axis=1)
test = pd.concat([test.drop('category_code', axis=1), test_encoded], axis=1)

In [60]:
# Engenharia de features adicionais para aumentar a acurácia
train['funding_per_round'] = train['funding_total_usd'] / (train['funding_rounds'] + 1)
test['funding_per_round'] = test['funding_total_usd'] / (test['funding_rounds'] + 1)
train['milestones_per_round'] = train['milestones'] / (train['funding_rounds'] + 1)
test['milestones_per_round'] = test['milestones'] / (test['funding_rounds'] + 1)
train['vc_software'] = train['has_VC'] * train['is_software']
test['vc_software'] = test['has_VC'] * test['is_software']
train['angel_software'] = train['has_angel'] * train['is_software']
test['angel_software'] = test['has_angel'] * test['is_software']

In [61]:
# Feature engineering adicional: log-transform para funding_total_usd e milestones
train['log_funding_total_usd'] = np.log1p(train['funding_total_usd'])
test['log_funding_total_usd'] = np.log1p(test['funding_total_usd'])
train['log_milestones'] = np.log1p(train['milestones'])
test['log_milestones'] = np.log1p(test['milestones'])

In [62]:
# Seleção de features relevantes (incluindo variáveis log-transformadas e de engenharia de features)
selected_features = [
    'funding_total_usd', 'relationships', 'is_software', 'funding_rounds', 'milestones',
    'avg_participants', 'has_VC', 'has_angel', 'has_roundA', 'has_roundB', 'has_roundC', 'has_roundD',
    'funding_per_round', 'milestones_per_round', 'vc_software', 'angel_software',
    'log_funding_total_usd', 'log_milestones',
    # Adicione outras dummies e colunas de categoria se necessário
    # Inclui todas as colunas one-hot de category_code
    *[col for col in train.columns if col.startswith('category_code_')]
 ]
X = train[selected_features]
y = train['labels']
X_test = test[selected_features]

In [63]:
# Padronização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

In [64]:
# Separação em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [70]:
# Treinamento do modelo Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)
# Avaliação inicial
y_pred = model.predict(X_val)
print('Acurácia:', accuracy_score(y_val, y_pred))
print('Precisão:', precision_score(y_val, y_pred))
print('Recall:', recall_score(y_val, y_pred))
print('F1-score:', f1_score(y_val, y_pred))
print('Matriz de Confusão:\n', confusion_matrix(y_val, y_pred))

Acurácia: 0.7538461538461538
Precisão: 0.7403846153846154
Recall: 0.9390243902439024
F1-score: 0.8279569892473119
Matriz de Confusão:
 [[21 27]
 [ 5 77]]
Matriz de Confusão:
 [[21 27]
 [ 5 77]]


In [71]:
# Geração do arquivo de submissão
test_pred = best_model.predict(X_test_scaled)
submission = sample_submission.copy()
submission['labels'] = test_pred
submission.to_csv('submission.csv', index=False)
print('Arquivo de submissão gerado: submission.csv')

Arquivo de submissão gerado: submission.csv
